In [15]:
import requests
import pyarrow as pa
import pyarrow.fs
import io

In [16]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 10209435971 (9.51 GB)
DFS Remaining: 9939046400 (9.26 GB)
DFS Used: 270389571 (257.86 MB)
DFS Used%: 2.65%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 78
	Missing blocks (with replication factor 1): 78
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.5:9866 (project-4-lukee4-dn-2.project-4-lukee4_default)
Hostname: 3c093826594b
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 270389571 (257.86 MB)
Non DFS Used: 15594839741 (14.52 GB)
DFS Remaining: 9939046400 (9.26 GB)
DFS Used%: 1.05%
DFS Remainin

In [17]:
#q9
r1 = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
if(r1.status_code == 403):
    s1 = 'lost'
else:
    j1 = r1.json()['Location']
    s1 = j1[7:19]

r2 = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=1048576&noredirect=true")
if(r2.status_code == 403):
    s2 = 'lost'
else:
    j2 = r2.json()['Location']
    s2 = j2[7:19]

count1 = 0
count2 = 0

for x in range(0, 174944099, 1048576):
    r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=" + str(x) + "&noredirect=true")
    if(r.status_code == 403):
        if(s1 == 'lost'):
            count1 += 1
        if(s2 == 'lost'):
            count2 += 1
    else:
        j = r.json()['Location']
        if(s1 in j):
            count1 += 1
        if(s2 in j):
            count2 += 1

Dict = {s1: count1, s2: count2}
Dict


{'lost': 78, '3c093826594b': 89}

In [18]:
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

In [19]:
healthyBlocks = []
for x in range(0, 174944099, 1048576):
    r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=" + str(x) + "&noredirect=true")
    if(r.status_code == 200):
        healthyBlocks.append(x)
healthyBlocks

[1048576,
 2097152,
 3145728,
 5242880,
 6291456,
 9437184,
 11534336,
 13631488,
 14680064,
 15728640,
 18874368,
 20971520,
 27262976,
 28311552,
 30408704,
 33554432,
 34603008,
 35651584,
 36700160,
 40894464,
 41943040,
 44040192,
 45088768,
 47185920,
 48234496,
 53477376,
 54525952,
 57671680,
 60817408,
 61865984,
 65011712,
 68157440,
 70254592,
 73400320,
 75497472,
 77594624,
 78643200,
 80740352,
 82837504,
 83886080,
 85983232,
 87031808,
 88080384,
 90177536,
 92274688,
 93323264,
 95420416,
 97517568,
 98566144,
 99614720,
 100663296,
 102760448,
 104857600,
 105906176,
 106954752,
 109051904,
 112197632,
 113246208,
 114294784,
 115343360,
 116391936,
 117440512,
 118489088,
 124780544,
 126877696,
 132120576,
 134217728,
 135266304,
 141557760,
 142606336,
 143654912,
 144703488,
 146800640,
 150994944,
 152043520,
 154140672,
 155189248,
 156237824,
 157286400,
 158334976,
 159383552,
 160432128,
 161480704,
 162529280,
 164626432,
 165675008,
 167772160,
 170917888,


In [23]:
#f = hdfs.open_input_file("/single.csv")
#f.read_at(1048576, 2)

In [21]:
#q10